# Ružička: Authorship Verification in Python

In [44]:
import logging

logging.basicConfig(level="INFO")
logger = logging.getLogger("ruzicka")

In [45]:
import numpy as np
from sklearn.pipeline import make_pipeline, Pipeline
from ruzicka.Order2Verifier import Order2Verifier
from ruzicka.utilities import *
from sklearn.preprocessing import LabelEncoder
from ruzicka.score_shifting import ScoreShifter
from ruzicka.evaluation import pan_metrics
from ruzicka.BDIVerifier import BDIVerifier
from typing import Union
import pandas as pd

In [63]:
PREFIX = "../data/2014/"
CORPORA = [
    "du_essays",
    "du_reviews",
    "en_essays",
    "en_novels",
    "gr_articles",
    "sp_articles",
]

In [64]:
def fit_shifter(
    corpus: str,
    vectorizer: Pipeline,
    verifier: Union[BDIVerifier, Order2Verifier],
    min_spread: float = 0.0,
) -> ScoreShifter:
    logger.info("Fitting shifter")
    train_data, test_data = load_pan_dataset(os.path.join(PREFIX, corpus, "train"))
    train_labels, train_documents = zip(*train_data)
    test_labels, test_documents = zip(*test_data)
    train_X = vectorizer.fit_transform(train_documents)
    test_X = vectorizer.transform(test_documents)
    label_encoder = LabelEncoder()
    label_encoder.fit(train_labels + test_labels)
    train_y = np.array(label_encoder.transform(train_labels), dtype="int")
    test_y = np.array(label_encoder.transform(test_labels), dtype="int")
    verifier.fit(train_X, train_y)
    test_scores = verifier.predict_proba(
        test_X=test_X, test_y=np.array(test_y), nb_imposters=30
    )
    gt_scores = load_ground_truth(
        filepath=os.path.join(PREFIX, corpus, "train", "truth.txt"), labels=test_labels
    )
    shifter = ScoreShifter(min=0.053, max=0.947, min_spread=min_spread)
    shifter.fit(predicted_scores=test_scores, ground_truth_scores=gt_scores)
    return shifter

In [65]:
def evaluate(corpus, vectorizer, verifier, shifter):
    logger.info("Loading evaluation corpus")
    train_data, test_data = load_pan_dataset(os.path.join(PREFIX, corpus, "test"))
    train_labels, train_documents = zip(*train_data)
    test_labels, test_documents = zip(*test_data)
    train_X = vectorizer.fit_transform(train_documents)
    test_X = vectorizer.transform(test_documents)
    label_encoder = LabelEncoder()
    label_encoder.fit(train_labels + test_labels)
    train_y = np.array(label_encoder.transform(train_labels), dtype="int")
    test_y = np.array(label_encoder.transform(test_labels), dtype="int")
    verifier.fit(train_X, train_y)
    test_scores = verifier.predict_proba(
        test_X=test_X, test_y=np.array(test_y), nb_imposters=30
    )
    gt_scores = load_ground_truth(
        filepath=os.path.join(PREFIX, corpus, "test", "truth.txt"), labels=test_labels
    )
    unanswered = (
        (shifter.optimal_p1 < test_scores) & (test_scores < shifter.optimal_p2)
    ).sum()
    high_conf = ((0.055 > test_scores) | (test_scores > 0.945)).sum()
    test_scores = np.array(shifter.transform(test_scores))
    badness = np.abs(test_scores - gt_scores).sum()
    fps = ((test_scores - gt_scores) > 0.5).sum()
    acc_score, auc_score, c_at_1_score = pan_metrics(
        prediction_scores=test_scores, ground_truth_scores=gt_scores
    )

    logger.info(f"Unanswered: {unanswered}")
    logger.info(f"High Confidence: {high_conf}")
    logger.info(f"False Positives: {fps}")
    logger.info(f"Badness: {badness:.3f}")
    logger.info(f"Accuracy: {acc_score:.3f}")
    logger.info(f"AUC: {auc_score:.3f}")
    logger.info(f"c@1: {c_at_1_score:.3f}")
    logger.info(f"AUC x c@1: {auc_score * c_at_1_score:.3f}")
    return (
        {
            "test_size": len(test_scores),
            "unanswed": unanswered,
            "high_conf": high_conf,
            "badness": badness,
            "p1": shifter.optimal_p1,
            "p2": shifter.optimal_p2,
            "false_positives": fps,
            "accuracy": acc_score,
            "auc": auc_score,
            "c@1": c_at_1_score,
            "final_score": auc_score * c_at_1_score,
        },
        (test_scores, gt_scores),
    )

In [52]:
vectorizer_35 = make_pipeline(
    TfidfVectorizer(
        sublinear_tf=True,
        use_idf=False,
        norm="l2",
        analyzer="char",
        ngram_range=(3, 5),
        max_features=10000,
    ),
    StandardScaler(with_mean=False),
    FunctionTransformer(lambda x: x.todense(), accept_sparse=True),
)

vectorizer_24 = make_pipeline(
    TfidfVectorizer(
        sublinear_tf=True,
        use_idf=False,
        norm="l2",
        analyzer="char",
        ngram_range=(2, 4),
        max_features=10000,
    ),
    StandardScaler(with_mean=False),
    FunctionTransformer(lambda x: x.todense(), accept_sparse=True),
)

vectorizer_25 = make_pipeline(
    TfidfVectorizer(
        sublinear_tf=True,
        use_idf=False,
        norm="l2",
        analyzer="char",
        ngram_range=(2, 5),
        max_features=10000,
    ),
    StandardScaler(with_mean=False),
    FunctionTransformer(lambda x: x.todense(), accept_sparse=True),
)

vecs = [
    ("2,3,4-grams", vectorizer_24),
    ("2,3,4,5-grams", vectorizer_25),
]

In [53]:
o2v_mm = Order2Verifier(
    metric="minmax", base="instance", rank=True, nb_bootstrap_iter=500, rnd_prop=0.5
)
o2v_cs = Order2Verifier(
    metric="cosine", base="instance", rank=True, nb_bootstrap_iter=500, rnd_prop=0.5
)
bdi_mm = BDIVerifier(metric="minmax", nb_bootstrap_iter=500, rnd_prop=0.33)
bdi_cs = BDIVerifier(metric="cosine", nb_bootstrap_iter=500, rnd_prop=0.33)

classifs = [
    ("Kestemont GI, Minmax", o2v_mm),
    ("Kestemont GI, Cosine", o2v_cs),
    ("BDI, Minmax", bdi_mm),
    ("BDI, Cosine", bdi_cs),
]

In [ ]:
results = []
global_results = []
for classif_name, classifier in classifs:
    for vec_name, vectorizer in vecs:
        globals_m = ([], [])
        globals_f = ([], [])
        for corpus in CORPORA:
            result_dict, tgt_m = evaluate(
                corpus, vectorizer, classifier, ScoreShifter().manual_fit(0.11, 0.89)
            )
            result_dict["classifier"] = f"{classif_name} + {vec_name} + manual"
            result_dict["corpus"] = corpus
            results.append(result_dict)
            globals_m[0].extend(tgt_m[0])
            globals_m[1].extend(tgt_m[1])
            fitted, tgt_f = evaluate(
                corpus,
                vectorizer,
                classifier,
                fit_shifter(corpus, vectorizer, classifier, min_spread=0.0),
            )
            globals_f[0].extend(tgt_f[0])
            globals_f[1].extend(tgt_f[1])
            fitted["classifier"] = f"{classif_name} + {vec_name} + fitted"
            fitted["corpus"] = corpus
            results.append(fitted)
        fitted_pan = pan_metrics(*globals_f)
        manual_pan = pan_metrics(*globals_m)
        global_results.append(
            {
                "classif": classif_name,
                "vec": vec_name,
                "shifter": "manual",
                "final": manual_pan[0],
                "auc": manual_pan[1],
                "c@1": manual_pan[2],
            }
        )
        global_results.append(
            {
                "classif": classif_name,
                "vec": vec_name,
                "shifter": "fitted",
                "final": fitted_pan[0],
                "auc": fitted_pan[1],
                "c@1": fitted_pan[2],
            }
        )

In [55]:
global_df = pd.DataFrame(global_results)
global_df = global_df.rename(columns={"final": "acc"})
global_df["final"] = global_df["auc"] * global_df["c@1"]
global_df

,classif,vec,shifter,acc,auc,c@1,final
0,"Kestemont GI, Minmax","2,3,4-grams",manual,0.628686,0.707473,0.572086,0.404735
1,"Kestemont GI, Minmax","2,3,4-grams",fitted,0.667560,0.759295,0.694336,0.527206
2,"Kestemont GI, Minmax","2,3,4,5-grams",manual,0.632708,0.705636,0.568043,0.400832
3,"Kestemont GI, Minmax","2,3,4,5-grams",fitted,0.660858,0.772510,0.702476,0.542669
4,"Kestemont GI, Cosine","2,3,4-grams",manual,0.613941,0.684785,0.536524,0.367404
5,"Kestemont GI, Cosine","2,3,4-grams",fitted,0.664879,0.755022,0.678076,0.511962
6,"Kestemont GI, Cosine","2,3,4,5-grams",manual,0.620643,0.698266,0.556847,0.388827
7,"Kestemont GI, Cosine","2,3,4,5-grams",fitted,0.672922,0.767870,0.694848,0.533553
8,"BDI, Minmax","2,3,4-grams",manual,0.684987,0.723490,0.652281,0.471919
9,"BDI, Minmax","2,3,4-grams",fitted,0.682306,0.730092,0.694406,0.506980


In [57]:
print(
    global_df.groupby(["classif", "vec", "shifter"])["acc", "auc", "c@1", "final"]
    .agg("mean")
    .to_latex(float_format="%.3f")
)

\begin{tabular}{lllrrrr}
\toprule
                     &             &        &   acc &   auc &   c@1 &  final \\
classif & vec & shifter &       &       &       &        \\
\midrule
BDI, Cosine & 2,3,4,5-grams & fitted & 0.681 & 0.727 & 0.694 &  0.505 \\
                     &             & manual & 0.672 & 0.715 & 0.649 &  0.464 \\
                     & 2,3,4-grams & fitted & 0.686 & 0.723 & 0.689 &  0.499 \\
                     &             & manual & 0.681 & 0.715 & 0.645 &  0.461 \\
BDI, Minmax & 2,3,4,5-grams & fitted & 0.689 & 0.731 & 0.695 &  0.508 \\
                     &             & manual & 0.682 & 0.726 & 0.660 &  0.479 \\
                     & 2,3,4-grams & fitted & 0.682 & 0.730 & 0.694 &  0.507 \\
                     &             & manual & 0.685 & 0.723 & 0.652 &  0.472 \\
Kestemont GI, Cosine & 2,3,4,5-grams & fitted & 0.673 & 0.768 & 0.695 &  0.534 \\
                     &             & manual & 0.621 & 0.698 & 0.557 &  0.389 \\
                     & 2,3,4-

/var/folders/wt/_hvfndq56jvd5k42nrkq9fy00000gn/T/ipykernel_72618/2276559794.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  global_df.groupby(["classif", "vec", "shifter"])["acc", "auc", "c@1", "final"]
/var/folders/wt/_hvfndq56jvd5k42nrkq9fy00000gn/T/ipykernel_72618/2276559794.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  global_df.groupby(["classif", "vec", "shifter"])["acc", "auc", "c@1", "final"]


In [71]:
# df = pd.DataFrame(results)
# df[["classif", "vec", "shifter"]] = df["classifier"].str.split("+", expand=True)
# df = df.drop("classifier", axis=1)

In [212]:
df = pd.read_csv("bdi_evaluation_newshifter.csv", index_col=0)
df

,test_size,unanswed,high_conf,badness,p1,p2,false_positives,accuracy,auc,c@1,final_score,corpus,classif,vec,shifter
0,96,29,50,25.961086,0.11000,0.89000,2,0.739583,0.938802,0.881619,0.827666,du_essays,"Kestemont GI, Minmax","2,3,4-grams",manual
1,96,10,51,15.402530,0.36590,0.65198,4,0.875000,0.962240,0.943142,0.907529,du_essays,"Kestemont GI, Minmax","2,3,4-grams",fitted
2,50,28,9,21.839900,0.11000,0.89000,0,0.660000,0.727200,0.499200,0.363018,du_reviews,"Kestemont GI, Minmax","2,3,4-grams",manual
3,50,3,9,20.437122,0.08876,0.10664,11,0.680000,0.742400,0.678400,0.503644,du_reviews,"Kestemont GI, Minmax","2,3,4-grams",fitted
4,200,148,19,94.892606,0.11000,0.89000,2,0.570000,0.587050,0.295800,0.173649,en_essays,"Kestemont GI, Minmax","2,3,4-grams",manual
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,200,3,161,82.323204,0.05300,0.06194,9,0.615000,0.616750,0.614075,0.378731,en_novels,"BDI, Cosine","2,3,4,5-grams",fitted
92,100,37,58,31.316047,0.11000,0.89000,3,0.790000,0.850400,0.671300,0.570874,gr_articles,"BDI, Cosine","2,3,4,5-grams",manual
93,100,1,57,24.461978,0.07982,0.16922,10,0.790000,0.877000,0.787800,0.690901,gr_articles,"BDI, Cosine","2,3,4,5-grams",fitted
94,100,40,52,28.710825,0.11000,0.89000,1,0.750000,0.867000,0.770000,0.667590,sp_articles,"BDI, Cosine","2,3,4,5-grams",manual


In [74]:
from IPython.display import display

In [214]:
for name, group in df.drop(["badness"], axis=1).groupby(["classif", "vec", "shifter"]):
    print(
        f"{''.join(name)}: Accuracy {group.accuracy.mean()*100:.2f} "
        f"C@1: {group['c@1'].mean()*100:.2f}  "
        f"Final: {group['final_score'].mean()*100:.2f}  "
        f"FP%: {group['false_positives'].sum()/group['test_size'].sum()*100:.3f} "
        f"Unanswered: {group['unanswed'].sum():>3} "
        f"({group['unanswed'].sum()/group['test_size'].sum()*100:.3f} %)"
    )
    display(group)

    # this_df = group[["corpus"] + [col for col in group.columns if col != "corpus"]]
    # print(
    #     this_df.drop(["badness", "p1", "p2", "classif", "vec", "shifter"], axis=1).to_latex(
    #         index=False, float_format="%.3f"
    #     )
    # )

BDI, Cosine  2,3,4,5-grams  fitted: Accuracy 69.60 C@1: 71.57  Final: 57.53  FP%: 6.032 Unanswered:  28 (3.753 %)


,test_size,unanswed,high_conf,p1,p2,false_positives,accuracy,auc,c@1,final_score,corpus,classif,vec,shifter
85,96,5,73,0.34802,0.58940,5,0.90625,0.962674,0.920573,0.886211,du_essays,"BDI, Cosine","2,3,4,5-grams",fitted
87,50,0,37,0.07088,0.09770,6,0.54000,0.697600,0.540000,0.376704,du_reviews,"BDI, Cosine","2,3,4,5-grams",fitted
89,200,5,167,0.06194,0.09770,6,0.59500,0.574800,0.599625,0.344664,en_essays,"BDI, Cosine","2,3,4,5-grams",fitted
91,200,3,161,0.05300,0.06194,9,0.61500,0.616750,0.614075,0.378731,en_novels,"BDI, Cosine","2,3,4,5-grams",fitted
93,100,1,57,0.07982,0.16922,10,0.79000,0.877000,0.787800,0.690901,gr_articles,"BDI, Cosine","2,3,4,5-grams",fitted
95,100,14,52,0.07088,0.33908,9,0.73000,0.930800,0.832200,0.774612,sp_articles,"BDI, Cosine","2,3,4,5-grams",fitted


BDI, Cosine  2,3,4,5-grams  manual: Accuracy 68.82 C@1: 67.03  Final: 52.89  FP%: 0.938 Unanswered: 157 (21.046 %)


,test_size,unanswed,high_conf,p1,p2,false_positives,accuracy,auc,c@1,final_score,corpus,classif,vec,shifter
84,96,19,73,0.11,0.89,2,0.854167,0.955078,0.923394,0.881914,du_essays,"BDI, Cosine","2,3,4,5-grams",manual
86,50,10,38,0.11,0.89,0,0.560000,0.665600,0.552000,0.367411,du_reviews,"BDI, Cosine","2,3,4,5-grams",manual
88,200,25,163,0.11,0.89,0,0.585000,0.615300,0.551250,0.339184,en_essays,"BDI, Cosine","2,3,4,5-grams",manual
90,200,26,162,0.11,0.89,1,0.590000,0.625700,0.553700,0.346450,en_novels,"BDI, Cosine","2,3,4,5-grams",manual
92,100,37,58,0.11,0.89,3,0.790000,0.850400,0.671300,0.570874,gr_articles,"BDI, Cosine","2,3,4,5-grams",manual
94,100,40,52,0.11,0.89,1,0.750000,0.867000,0.770000,0.667590,sp_articles,"BDI, Cosine","2,3,4,5-grams",manual


BDI, Cosine  2,3,4-grams  fitted: Accuracy 70.60 C@1: 71.20  Final: 57.77  FP%: 6.568 Unanswered:  43 (5.764 %)


...

,test_size,unanswed,high_conf,p1,p2,false_positives,accuracy,auc,c@1,final_score,corpus,classif,vec,shifter
12,96,29,54,0.11,0.89,2,0.750,0.943793,0.881619,0.832066,du_essays,"Kestemont GI, Minmax","2,3,4,5-grams",manual
14,50,24,14,0.11,0.89,0,0.640,0.708800,0.532800,0.377649,du_reviews,"Kestemont GI, Minmax","2,3,4,5-grams",manual
16,200,142,22,0.11,0.89,2,0.595,0.607950,0.342000,0.207919,en_essays,"Kestemont GI, Minmax","2,3,4,5-grams",manual
18,200,68,80,0.11,0.89,2,0.585,0.640500,0.529300,0.339017,en_novels,"Kestemont GI, Minmax","2,3,4,5-grams",manual
20,100,59,19,0.11,0.89,3,0.710,0.771200,0.588300,0.453697,gr_articles,"Kestemont GI, Minmax","2,3,4,5-grams",manual
22,100,57,28,0.11,0.89,1,0.610,0.844800,0.659400,0.557061,sp_articles,"Kestemont GI, Minmax","2,3,4,5-grams",manual


Kestemont GI, Minmax  2,3,4-grams  fitted: Accuracy 69.42 C@1: 72.08  Final: 57.87  FP%: 16.622 Unanswered: 123 (16.488 %)


,test_size,unanswed,high_conf,p1,p2,false_positives,accuracy,auc,c@1,final_score,corpus,classif,vec,shifter
1,96,10,51,0.36590,0.65198,4,0.875,0.96224,0.943142,0.907529,du_essays,"Kestemont GI, Minmax","2,3,4-grams",fitted
3,50,3,9,0.08876,0.10664,11,0.680,0.74240,0.678400,0.503644,du_reviews,"Kestemont GI, Minmax","2,3,4-grams",fitted
5,200,6,18,0.07088,0.07982,78,0.560,0.54385,0.571650,0.310892,en_essays,"Kestemont GI, Minmax","2,3,4-grams",fitted
7,200,60,85,0.07088,0.21392,13,0.580,0.64715,0.578500,0.374376,en_novels,"Kestemont GI, Minmax","2,3,4-grams",fitted
9,100,34,16,0.25862,0.75032,4,0.730,0.83620,0.750400,0.627484,gr_articles,"Kestemont GI, Minmax","2,3,4-grams",fitted
11,100,10,26,0.42848,0.61622,14,0.740,0.93220,0.803000,0.748557,sp_articles,"Kestemont GI, Minmax","2,3,4-grams",fitted


Kestemont GI, Minmax  2,3,4-grams  manual: Accuracy 64.49 C@1: 57.87  Final: 45.42  FP%: 1.475 Unanswered: 386 (51.743 %)


,test_size,unanswed,high_conf,p1,p2,false_positives,accuracy,auc,c@1,final_score,corpus,classif,vec,shifter
0,96,29,50,0.11,0.89,2,0.739583,0.938802,0.881619,0.827666,du_essays,"Kestemont GI, Minmax","2,3,4-grams",manual
2,50,28,9,0.11,0.89,0,0.660000,0.727200,0.499200,0.363018,du_reviews,"Kestemont GI, Minmax","2,3,4-grams",manual
4,200,148,19,0.11,0.89,2,0.570000,0.587050,0.295800,0.173649,en_essays,"Kestemont GI, Minmax","2,3,4-grams",manual
6,200,66,84,0.11,0.89,2,0.610000,0.643350,0.551950,0.355097,en_novels,"Kestemont GI, Minmax","2,3,4-grams",manual
8,100,59,16,0.11,0.89,3,0.690000,0.769600,0.588300,0.452756,gr_articles,"Kestemont GI, Minmax","2,3,4-grams",manual
10,100,56,24,0.11,0.89,2,0.600000,0.844400,0.655200,0.553251,sp_articles,"Kestemont GI, Minmax","2,3,4-grams",manual


In [66]:
def evaluate_one(
    vectorizer: Pipeline,
    classifier: Union[BDIVerifier, Order2Verifier],
    shifter_strategy: str = "manual",
    min_spread: float = 0.0,
):
    # Evaluate one classifier/vectorizer/shifter on all the corpora
    results = []
    globals = ([], [])
    for corpus in CORPORA:
        if shifter_strategy == "manual":
            result_dict, tgt = evaluate(
                corpus, vectorizer, classifier, ScoreShifter().manual_fit(0.11, 0.89)
            )
        else:
            result_dict, tgt = evaluate(
                corpus,
                vectorizer,
                classifier,
                fit_shifter(corpus, vectorizer, classifier, min_spread=min_spread),
            )
        result_dict["classifier"] = f"{classif_name} + {vec_name} + {shifter_strategy}"
        result_dict["corpus"] = corpus
        results.append(result_dict)
        globals[0].extend(tgt[0])
        globals[1].extend(tgt[1])
    return (results, globals)

In [67]:
r, g = evaluate_one(vectorizer_25, bdi_mm)

01/27/2024 12:36:16 [ruzicka:INFO] Loading evaluation corpus
01/27/2024 12:36:17 [ruzicka:INFO] Fitting on 191 documents...
01/27/2024 12:36:17 [ruzicka:INFO] Predicting on 96 documents
01/27/2024 12:36:38 [ruzicka:INFO] Unanswered: 19
01/27/2024 12:36:38 [ruzicka:INFO] High Confidence: 74
01/27/2024 12:36:38 [ruzicka:INFO] False Positives: 2
01/27/2024 12:36:38 [ruzicka:INFO] Badness: 13.888
01/27/2024 12:36:38 [ruzicka:INFO] Accuracy: 0.854
01/27/2024 12:36:38 [ruzicka:INFO] AUC: 0.956
01/27/2024 12:36:38 [ruzicka:INFO] c@1: 0.923
01/27/2024 12:36:38 [ruzicka:INFO] AUC x c@1: 0.883
01/27/2024 12:36:38 [ruzicka:INFO] Loading evaluation corpus
01/27/2024 12:36:38 [ruzicka:INFO] Fitting on 50 documents...
01/27/2024 12:36:38 [ruzicka:INFO] Predicting on 50 documents
01/27/2024 12:36:49 [ruzicka:INFO] Unanswered: 10
01/27/2024 12:36:49 [ruzicka:INFO] High Confidence: 34
01/27/2024 12:36:49 [ruzicka:INFO] False Positives: 0
01/27/2024 12:36:49 [ruzicka:INFO] Badness: 21.345
01/27/2024 12:

In [68]:
pd.DataFrame(r)

,test_size,unanswed,high_conf,badness,p1,p2,false_positives,accuracy,auc,c@1,final_score,classifier,corpus
0,96,19,74,13.888282,0.11,0.89,2,0.854167,0.956163,0.923394,0.882915,"BDI, Cosine + 2,3,4,5-grams + manual",du_essays
1,50,10,34,21.344779,0.11,0.89,0,0.580000,0.696800,0.576000,0.401357,"BDI, Cosine + 2,3,4,5-grams + manual",du_reviews
2,200,29,163,85.001032,0.11,0.89,0,0.615000,0.631350,0.572500,0.361448,"BDI, Cosine + 2,3,4,5-grams + manual",en_essays
3,200,34,162,88.390146,0.11,0.89,0,0.595000,0.633100,0.555750,0.351845,"BDI, Cosine + 2,3,4,5-grams + manual",en_novels
4,100,33,62,32.638649,0.11,0.89,3,0.800000,0.826400,0.691600,0.571538,"BDI, Cosine + 2,3,4,5-grams + manual",gr_articles
5,100,35,57,27.471932,0.11,0.89,1,0.780000,0.883200,0.810000,0.715392,"BDI, Cosine + 2,3,4,5-grams + manual",sp_articles


In [69]:
pan_metrics(*g)

(0.6849865951742627, 0.7245326279927262, 0.659334143133351)

In [70]:
len(g[0])

746

In [107]:
gdf = pd.DataFrame(g).T
gdf.columns = ["proba", "gt"]
gdf["classif"] = "BDI"
gdf

,proba,gt,classif
0,0.999995,1.0,BDI
1,0.999995,1.0,BDI
2,0.999995,1.0,BDI
3,0.999995,1.0,BDI
4,0.000000,0.0,BDI
...,...,...,...
741,0.972778,1.0,BDI
742,0.009073,0.0,BDI
743,0.800397,1.0,BDI
744,0.009073,0.0,BDI


In [74]:
def tfu(r):
    if r["proba"] == 0.5:
        return "U"
    elif (r["proba"] < 0.5) and r["gt"] == 0:
        return "T"
    elif (r["proba"] > 0.5) and r["gt"] == 1:
        return "T"
    else:
        return "F"

In [108]:
gdf["tfu"] = gdf.apply(tfu, axis=1)
gdf

,proba,gt,classif,tfu
0,0.999995,1.0,BDI,T
1,0.999995,1.0,BDI,T
2,0.999995,1.0,BDI,T
3,0.999995,1.0,BDI,T
4,0.000000,0.0,BDI,T
...,...,...,...,...
741,0.972778,1.0,BDI,T
742,0.009073,0.0,BDI,T
743,0.800397,1.0,BDI,T
744,0.009073,0.0,BDI,T


In [118]:
kr, kg = evaluate_one(vectorizer_25, o2v_mm, shifter_strategy="fitted")

01/27/2024 01:25:43 [ruzicka:INFO] Fitting shifter
01/27/2024 01:25:44 [ruzicka:INFO] Fitting on 172 documents in instance mode...
01/27/2024 01:25:44 [ruzicka:INFO] Predicting on 96 documents
01/27/2024 01:26:09 [ruzicka:INFO] p1 for optimal combo: 0.268
01/27/2024 01:26:09 [ruzicka:INFO] p2 for optimal combo: 0.795
01/27/2024 01:26:09 [ruzicka:INFO] AUC for optimal combo: 94.99%
01/27/2024 01:26:09 [ruzicka:INFO] c@1 for optimal combo: 95.35%
01/27/2024 01:26:09 [ruzicka:INFO] Loading evaluation corpus
01/27/2024 01:26:10 [ruzicka:INFO] Fitting on 191 documents in instance mode...
01/27/2024 01:26:10 [ruzicka:INFO] Predicting on 96 documents
01/27/2024 01:26:32 [ruzicka:INFO] Unanswered: 16
01/27/2024 01:26:32 [ruzicka:INFO] High Confidence: 54
01/27/2024 01:26:32 [ruzicka:INFO] False Positives: 2
01/27/2024 01:26:32 [ruzicka:INFO] Badness: 17.750
01/27/2024 01:26:32 [ruzicka:INFO] Accuracy: 0.854
01/27/2024 01:26:32 [ruzicka:INFO] AUC: 0.965
01/27/2024 01:26:32 [ruzicka:INFO] c@1: 0

In [119]:
kdf = pd.DataFrame(kg).T
kdf.columns = ["proba", "gt"]
kdf["classif"] = "O2V"
kdf["tfu"] = kdf.apply(tfu, axis=1)
kdf

,proba,gt,classif,tfu
0,0.999998,1.0,O2V,T
1,0.999998,1.0,O2V,T
2,0.997563,1.0,O2V,T
3,0.999998,1.0,O2V,T
4,0.101527,0.0,O2V,T
...,...,...,...,...
741,0.974287,1.0,O2V,T
742,0.277101,0.0,O2V,T
743,0.886715,1.0,O2V,T
744,0.255455,0.0,O2V,T


In [120]:
rdf = pd.concat([gdf, kdf]).reset_index(drop=True)

In [160]:
acc, auc, c1 = pan_metrics(*g)
print(f"Accuracy: {acc:.2%} AUC {auc:.2f} PAN Score (AUC * C@1): {auc*c1:.2%}")

Accuracy: 68.50% AUC 0.72 PAN Score (AUC * C@1): 47.77%


In [197]:
acc, auc, c1 = pan_metrics(*kg)
print(f"Accuracy: {acc:.2%} AUC {auc:.2f} PAN Score (AUC * C@1): {auc*c1:.2%}")

Accuracy: 67.02% AUC 0.76 PAN Score (AUC * C@1): 52.89%


In [202]:
%%R -i rdf -h 4.5 -w 8 -u in -r 300
library(ggridges)
library(paletteer)
library(ggplot2)
library(dplyr)
library(showtext)
font_add_google("Roboto Condensed", "fnt")
showtext_auto()

plt <- "ggsci::category10_d3"
fgcol <- '#1e1a1d'
bgcol <- 'white'
plt <- c('green', 'red', 'lightblue')
ggplot(data=rdf,aes(proba,y=classif,fill=tfu,color=tfu, shape=tfu)) +
coord_cartesian(clip = "off") +
geom_jitter(data=filter(rdf,tfu=='T'),size=3.2, stroke=1.5, alpha=0.8, width=0.0, height=0.25, color='#79f059',shape=16) +
geom_jitter(data=filter(rdf,tfu=='F'),size=2.8, stroke=1.0, alpha=0.9, width=0.0, height=0.25,color='red',shape=4) +
annotate('text', hjust=0,x=-0.01,y=2.37, label=c("Accuracy: 67.02% AUC 0.76 AUC × c@1: 52.89%"), color=fgcol,size=3) +
annotate('text',hjust=0,x=-0.01,y=1.37, label=c("Accuracy: 68.50% AUC 0.72 AUC × c@1: 47.77%"), color=fgcol, size=3) +

geom_vline(xintercept=0.5, colour=fgcol, linetype='dashed', size=0.3) +
scale_fill_manual(values=c('red','#63a152','lightblue'))+
scale_color_manual(values=c('red','#63a152','lightblue'))+
scale_shape_manual(values=c(4,16,1)) +
theme_bw() +
xlab("") +
ylab("") +
ggtitle("Correct / Incorrect results by reported probability (global micro-average)") +
theme(
    panel.border = element_blank(),
    legend.position='none',
    panel.background = element_rect(fill = bgcol,color=bgcol),
    plot.background = element_rect(fill = bgcol,color=bgcol),
    plot.title = element_text(hjust = 0.5, size=15, family="fnt", color=fgcol),
    axis.line.x=element_line(linewidth=0.3,color=fgcol),
    axis.line.y=element_line(linewidth=0.3,color=fgcol),
    axis.ticks.x=element_line(linewidth=0.2,color=fgcol),
    axis.ticks.y=element_line(linewidth=0.2,color=fgcol),
    axis.text.x=element_text(size=8, family="fnt", color=fgcol),
    axis.text.y=element_text(size=11, family="fnt", color=fgcol),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
)

# fn <- "~/papers/bdi-paper/paper/figures/bdi_o2v.pdf"
# ggsave(fn, dpi=600, width=8, height=4.5)
